In [0]:
%python
dbutils.fs.rm("dbfs:/user/hive/warehouse/qualified_country_list", True)
dbutils.fs.rm("dbfs:/user/hive/warehouse/matches", True)
dbutils.fs.rm("dbfs:/user/hive/warehouse/group_stage_ranking", True)
dbutils.fs.rm("dbfs:/user/hive/warehouse/round_of_16", True)
dbutils.fs.rm("dbfs:/user/hive/warehouse/eurocup_third_place_combi", True)
dbutils.fs.rm("dbfs:/user/hive/warehouse/eurocup_third_place_combi", True)
dbutils.fs.rm("dbfs:/user/hive/warehouse/eurocup_third_place_combi", True)

Out[1]: False

In [0]:
%sql
CREATE OR REPLACE TABLE qualified_country_list (
  TeamId INTEGER,
  TeamName VARCHAR(255),
  TeamGroup VARCHAR(255)
);

INSERT INTO qualified_country_list (TeamId, TeamName, TeamGroup)
VALUES
(1, 'France', 'A'),
(2, 'Italy', 'A'),
(3, 'Belgium', 'A'),
(4, 'Portugal', 'A'),
(5, 'Spain', 'B'),
(6, 'Netherlands', 'B'),
(7, 'Germany', 'B'),
(8, 'England', 'B'),
(9, 'Turkey', 'C'),
(10, 'Austria', 'C'),
(11, 'Switzerland', 'C'),
(12, 'Denmark', 'C'),
(13, 'Croatia', 'D'),
(14, 'Ukraine', 'D'),
(15, 'Sweden', 'D'),
(16, 'Poland', 'D'),
(17, 'Wales', 'E'),
(18, 'Czech Republic', 'E'),
(19, 'Slovakia', 'E'),
(20, 'Hungary', 'E'),
(21, 'Finland', 'F'),
(22, 'North Macedonia', 'F'),
(23, 'Scotland', 'F'),
(24, 'Russia', 'F');

SELECT * FROM qualified_country_list;

TeamId,TeamName,TeamGroup
1,France,A
2,Italy,A
3,Belgium,A
4,Portugal,A
5,Spain,B
6,Netherlands,B
7,Germany,B
8,England,B
9,Turkey,C
10,Austria,C


In [0]:
%sql
create or replace table matches(
  MatchId int,
  TeamGroup varchar(20),
  home_country varchar(20),
  away_country varchar (20),
  home_country_goal int,
  away_country_goal int,
  winner_country varchar(20)
)

In [0]:
%sql
INSERT INTO matches
SELECT DISTINCT
row_number() OVER (ORDER BY 1) as MATCHID,
t1.TeamGroup as TeamGroup,
t1.TeamName AS home_country,
t2.TeamName As away_country,
(SELECT FLOOR(RAND() * 6) + 1) AS home_country_goal,
(SELECT FLOOR(RAND() * 6) + 1) AS away_country_goal,
'Initially Draw' AS winner_country

FROM
qualified_country_list as t1
JOIN
qualified_country_list as t2
ON t1.TeamGroup = t2.TeamGroup AND t1.TeamId < t2.TeamId;


select * from matches;

MatchId,TeamGroup,home_country,away_country,home_country_goal,away_country_goal,winner_country
1,A,France,Portugal,3,4,Initially Draw
2,A,France,Belgium,3,3,Initially Draw
3,A,France,Italy,5,1,Initially Draw
4,A,Italy,Portugal,4,3,Initially Draw
5,A,Italy,Belgium,4,6,Initially Draw
6,A,Belgium,Portugal,4,3,Initially Draw
7,B,Spain,England,6,2,Initially Draw
8,B,Spain,Germany,4,3,Initially Draw
9,B,Spain,Netherlands,4,5,Initially Draw
10,B,Netherlands,England,1,3,Initially Draw


In [0]:
%sql
UPDATE matches SET winner_country = 
CASE
WHEN home_country_goal > away_country_goal THEN home_country
WHEN home_country_goal < away_country_goal THEN away_country
ELSE 'Draw'
END;

select * from matches;

MatchId,TeamGroup,home_country,away_country,home_country_goal,away_country_goal,winner_country
1,A,France,Portugal,3,4,Portugal
2,A,France,Belgium,3,3,Draw
3,A,France,Italy,5,1,France
4,A,Italy,Portugal,4,3,Italy
5,A,Italy,Belgium,4,6,Belgium
6,A,Belgium,Portugal,4,3,Belgium
7,B,Spain,England,6,2,Spain
8,B,Spain,Germany,4,3,Spain
9,B,Spain,Netherlands,4,5,Netherlands
10,B,Netherlands,England,1,3,England


In [0]:
%sql
CREATE OR REPLACE TABLE group_stage_ranking AS
SELECT
  team_id as TeamName,
  TeamGroup,
  COUNT(*) AS Matches_Played,
  SUM (
    CASE
      WHEN goals_scored > goals_allowed THEN 1
      ELSE 0
    END
  ) AS Matches_won,
  SUM (
    CASE
      WHEN goals_scored = goals_allowed THEN 1
      ELSE 0
    END
  ) AS Matches_draw,
  SUM(
    CASE
      WHEN goals_scored < goals_allowed THEN 1
      ELSE 0
    END
  ) AS Matches_lost,
  SUM(goals_scored) AS Goals_scored,
  SUM(goals_allowed) AS Goals_allowed,
  SUM(goals_scored) - SUM(goals_allowed) AS Goal_difference,
  SUM(
    CASE
      WHEN goals_scored > goals_allowed THEN 3
      WHEN goals_scored = goals_allowed THEN 1
      ELSE 0
    END
  ) AS points
FROM(
    SELECT
      TeamGroup,
      home_country AS team_id,
      home_country_goal AS goals_scored,
      away_country_goal AS goals_allowed
    FROM
      matches
    UNION ALL
    SELECT
      TeamGroup,
      away_country AS team_id,
      away_country_goal AS goals_scored,
      home_country_goal AS goals_allowed
    FROM
      matches
  ) AS group_matches
GROUP BY
  team_id,
  TeamGroup

ORDER BY
  TeamGroup,
  points DESC;

select * FROM group_stage_ranking;

TeamName,TeamGroup,Matches_Played,Matches_won,Matches_draw,Matches_lost,Goals_scored,Goals_allowed,Goal_difference,points
Belgium,A,3,2,1,0,13,10,3,7
France,A,3,1,1,1,11,8,3,4
Italy,A,3,1,0,2,9,14,-5,3
Portugal,A,3,1,0,2,10,11,-1,3
Spain,B,3,2,0,1,14,10,4,6
England,B,3,2,0,1,10,11,-1,6
Netherlands,B,3,1,0,2,8,12,-4,3
Germany,B,3,1,0,2,12,11,1,3
Denmark,C,3,3,0,0,13,5,8,9
Austria,C,3,1,1,1,15,16,-1,4


In [0]:
%sql
CREATE OR REPLACE TABLE round_of_16 AS
SELECT * FROM(
  SELECT TeamName, TeamGroup, points, Goal_difference,
  row_number() OVER (PARTITION BY TeamGroup ORDER BY points DESC, Goal_difference DESC, Goals_scored DESC, Matches_won DESC)AS ranks
  FROM group_stage_ranking
)AS top2_teams
WHERE ranks <= 2;

select * from round_of_16;

TeamName,TeamGroup,points,Goal_difference,ranks
Belgium,A,7,3,1
France,A,4,3,2
Spain,B,6,4,1
England,B,6,-1,2
Denmark,C,9,8,1
Austria,C,4,-1,2
Ukraine,D,7,5,1
Poland,D,5,4,2
Czech Republic,E,6,4,1
Slovakia,E,6,0,2


In [0]:
%sql
INSERT INTO round_of_16
SELECT * FROM(
  SELECT TeamName, TeamGroup, points, Goal_difference,
  row_number() OVER (PARTITION BY TeamGroup ORDER BY points DESC, Goal_difference DESC, Goals_scored DESC, Matches_won DESC) AS ranks
  FROM group_stage_ranking
)AS top2_teams
WHERE ranks = 3 LIMIT 4;

select * from round_of_16;

TeamName,TeamGroup,points,Goal_difference,ranks
Belgium,A,7,3,1
France,A,4,3,2
Spain,B,6,4,1
England,B,6,-1,2
Denmark,C,9,8,1
Austria,C,4,-1,2
Ukraine,D,7,5,1
Poland,D,5,4,2
Czech Republic,E,6,4,1
Slovakia,E,6,0,2


In [0]:
%sql
CREATE or replace  TABLE eurocup_third_place_combi(
  ID INTEGER,
  selected_groups VARCHAR(4),
  Team1B CHAR(1),
  Team1C CHAR(1),
  Team1E CHAR(1),
  Team1F CHAR(1)
);

-- Step 2: Insert the provided values into the "THIRDMATCHUP" table
INSERT INTO  eurocup_third_place_combi (ID, selected_groups, Team1B, Team1C, Team1E, Team1F) VALUES
(1, 'ABCD', 'A', 'D', 'B', 'C'),
(2, 'ABCE', 'A', 'E', 'B', 'C'),
(3, 'ABCF', 'A', 'F', 'B', 'C'),
(4, 'ABDE', 'D', 'E', 'A', 'B'),
(5, 'ABDF', 'D', 'F', 'A', 'B'),
(6, 'ABEF', 'E', 'F', 'B', 'A'),
(7, 'ACDE', 'E', 'D', 'C', 'A'),
(8, 'ACDF', 'F', 'D', 'C', 'A'),
(9, 'ACEF', 'E', 'F', 'C', 'A'),
(10, 'ADEF', 'E', 'F', 'D', 'A'),
(11, 'BCDE', 'E', 'D', 'B', 'C'),
(12, 'BCDF', 'F', 'D', 'C', 'B'),
(13, 'BCEF', 'F', 'E', 'C', 'B'),
(14, 'BDEF', 'F', 'E', 'D', 'B'),
(15, 'CDEF', 'F', 'E', 'D', 'C');

num_affected_rows,num_inserted_rows
15,15


In [0]:
%sql




In [0]:
%sql
INSERT INTO matches (MatchID, TeamGroup, home_country, away_country, home_country_goal, away_country_goal, winner_country)
WITH combination AS (
  SELECT concat_ws('', sort_array(collect_list(TeamGroup)))AS group_combination
  FROM round_of_16
  WHERE ranks = 3
),
keys AS (
  SELECT * FROM eurocup_third_place_combi
  WHERE selected_groups = (SELECT group_combination FROM combination)
)
SELECT
row_number() OVER (ORDER BY 1) + 36 AS MatchId,
'N' AS Team_group,
r1.TeamName AS home_country,
r2.TeamName AS away_country,
CAST(rand() * 6 AS INTEGER) AS home_country_goal,
CAST(rand() * 6 AS INTEGER) AS away_country_goal,
'None' AS match_result
FROM round_of_16 r1
JOIN round_of_16 r2 ON r1.TeamGroup = 'A' AND r1.ranks = 1 AND r2.TeamGroup = 'C' AND r2.ranks = 2


UNION ALL

SELECT 
    row_number() OVER (ORDER BY 1) + 37 AS match_id,
    'N' AS group_name,
    r1.TeamName AS home_country,
    r2.TeamName AS away_country,
    CAST(rand() * 6 AS INT) AS home_country_goal, 
    CAST(rand() * 6 AS INT) AS away_country_goal,
    'NONE' AS match_result
FROM 
    round_of_16 r1
JOIN 
 round_of_16 r2 ON r1.TeamGroup = 'A' AND r1.ranks = 2 AND r2.TeamGroup = 'B' AND r2.ranks = 2
   

UNION ALL

SELECT 
    row_number() OVER (ORDER BY 1) + 38 AS match_id,
    'N' AS group_name,
    r1.TeamName AS home_country,
    r2.TeamName AS away_country,
    CAST(rand() * 6 AS INT) AS home_country_goal, 
    CAST(rand() * 6 AS INT) AS away_country_goal,
    'NONE' AS match_result
FROM 
    round_of_16 r1
JOIN 
    round_of_16 r2 ON r1.TeamGroup = 'B' AND r1.ranks = 1 AND r2.TeamGroup = (SELECT TEAM1B FROM keys) AND r2.ranks = 3

UNION ALL

SELECT 
    row_number() OVER (ORDER BY 1) + 39 AS match_id,
    'N' AS group_name,
    r1.TeamName AS home_country,
    r2.TeamName AS away_country,
    CAST(rand() * 6 AS INT) AS home_country_goal, 
    CAST(rand() * 6 AS INT) AS away_country_goal,
    'NONE' AS match_result
FROM 
    round_of_16 r1
JOIN 
    round_of_16 r2 ON r1.TeamGroup = 'C' AND r1.ranks = 1 AND r2.TeamGroup = (select Team1C from keys) AND r2.ranks = 3

UNION ALL

SELECT 
    Row_Number() OVER (ORDER BY 1) + 40 AS match_id,
    'N' AS group_name,
    r1.TeamName AS home_country,
    r2.TeamName AS away_country,
    CAST(rand() * 6 AS INT) AS home_country_goal, 
    CAST(rand() * 6 AS INT) AS away_country_goal,
    'NONE' AS match_result
FROM 
    round_of_16 r1
JOIN 
     round_of_16 r2 ON r1.TeamGroup = 'F' AND r1.ranks = 1 AND r2.TeamGroup = (select Team1F from keys) AND r2.ranks = 3


UNION ALL

SELECT 
    Row_Number() OVER (ORDER BY 1) + 41 AS match_id,
    'N' AS group_name,
    r1.TeamName AS home_country,
    r2.TeamName AS away_country,
    CAST(rand() * 6 AS INT) AS home_country_goal, 
    CAST(rand() * 6 AS INT) AS away_country_goal,
    'NONE' AS match_result
FROM 
    round_of_16 r1
JOIN 
    round_of_16 r2 ON r1.TeamGroup = 'D' AND r1.ranks = 2 AND r2.TeamGroup = 'E' AND r2.ranks = 2
   

UNION ALL

SELECT 
    Row_Number() OVER (ORDER BY 1) + 42 AS match_id,
    'N' AS group_name,
    r1.TeamName AS home_country,
    r2.TeamName AS away_country,
    CAST(rand() * 6 AS INT) AS home_country_goal, 
    CAST(rand() * 6 AS INT) AS away_country_goal,
    'NONE' AS match_result
FROM 
    round_of_16 r1
JOIN 
    round_of_16 r2 ON r1.TeamGroup = 'E' AND r1.ranks = 1 AND r2.TeamGroup = (SELECT Team1E from keys) AND r2.ranks = 3
   

UNION ALL

SELECT 
    Row_Number() OVER (ORDER BY 1) + 43 AS match_id,
    'N' AS group_name,
    r1.TeamName AS home_country,
    r2.TeamName AS away_country,
    CAST(rand() * 6 AS INT) AS home_country_goal, 
    CAST(rand() * 6 AS INT) AS away_country_goal,
    'NONE' AS match_result
FROM 
    round_of_16 r1
JOIN 
    round_of_16 r2 ON r1.TeamGroup = 'D' AND r1.ranks = 2 AND r2.TeamGroup = 'F' AND r2.ranks = 2;

num_affected_rows,num_inserted_rows
8,8


In [0]:
%sql
select * from matches order by MatchId;

MatchId,TeamGroup,home_country,away_country,home_country_goal,away_country_goal,winner_country
1,A,France,Portugal,3,4,Portugal
2,A,France,Belgium,3,3,Draw
3,A,France,Italy,5,1,France
4,A,Italy,Portugal,4,3,Italy
5,A,Italy,Belgium,4,6,Belgium
6,A,Belgium,Portugal,4,3,Belgium
7,B,Spain,England,6,2,Spain
8,B,Spain,Germany,4,3,Spain
9,B,Spain,Netherlands,4,5,Netherlands
10,B,Netherlands,England,1,3,England


In [0]:
%sql
UPDATE matches
SET winner_country = 
CASE
WHEN home_country_goal > away_country_goal THEN home_country
WHEN home_country_goal < away_country_goal THEN away_country
ELSE
CASE
WHEN (home_country_goal + 1) % 2 = 1 THEN home_country
ELSE away_country
END
END

num_affected_rows
44


In [0]:
%sql
select * from matches order by MatchId;

MatchId,TeamGroup,home_country,away_country,home_country_goal,away_country_goal,winner_country
1,A,France,Portugal,3,4,Portugal
2,A,France,Belgium,3,3,Belgium
3,A,France,Italy,5,1,France
4,A,Italy,Portugal,4,3,Italy
5,A,Italy,Belgium,4,6,Belgium
6,A,Belgium,Portugal,4,3,Belgium
7,B,Spain,England,6,2,Spain
8,B,Spain,Germany,4,3,Spain
9,B,Spain,Netherlands,4,5,Netherlands
10,B,Netherlands,England,1,3,England


Quarter Finals

In [0]:
%sql
INSERT INTO matches (MatchID, TeamGroup, home_country, away_country, home_country_goal, away_country_goal, winner_country)

SELECT
row_number() OVER (ORDER BY 1) + 44 AS MatchId,
'N' AS Team_group,
r1.winner_country AS home_country,
r2.winner_country AS away_country,
CAST(rand() * 6 AS INTEGER) AS home_country_goal,
CAST(rand() * 6 AS INTEGER) AS away_country_goal,
'None' AS match_result

FROM 
    matches r1
JOIN 
    matches r2 ON 
        r1.MatchId = 37 AND r2.MatchId = 38

UNION ALL

SELECT
row_number() OVER (ORDER BY 1) + 45 AS MatchId,
'N' AS Team_group,
r1.winner_country AS home_country,
r2.winner_country AS away_country,
CAST(rand() * 6 AS INTEGER) AS home_country_goal,
CAST(rand() * 6 AS INTEGER) AS away_country_goal,
'None' AS match_result

FROM 
    matches r1
JOIN 
    matches r2 ON 
        r1.MatchId = 39 AND r2.MatchId = 40

UNION ALL

SELECT
row_number() OVER (ORDER BY 1) + 46 AS MatchId,
'N' AS Team_group,
r1.winner_country AS home_country,
r2.winner_country AS away_country,
CAST(rand() * 6 AS INTEGER) AS home_country_goal,
CAST(rand() * 6 AS INTEGER) AS away_country_goal,
'None' AS match_result

FROM 
    matches r1
JOIN 
    matches r2 ON 
        r1.MatchId = 41 AND r2.MatchId = 42

UNION ALL

SELECT
row_number() OVER (ORDER BY 1) + 47 AS MatchId,
'N' AS Team_group,
r1.winner_country AS home_country,
r2.winner_country AS away_country,
CAST(rand() * 6 AS INTEGER) AS home_country_goal,
CAST(rand() * 6 AS INTEGER) AS away_country_goal,
'None' AS match_result

FROM 
    matches r1
JOIN 
    matches r2 ON 
        r1.MatchId = 43 AND r2.MatchId = 44    


num_affected_rows,num_inserted_rows
4,4


In [0]:
%sql
select * from matches order by MatchId;

MatchId,TeamGroup,home_country,away_country,home_country_goal,away_country_goal,winner_country
1,A,France,Portugal,3,4,Portugal
2,A,France,Belgium,3,3,Belgium
3,A,France,Italy,5,1,France
4,A,Italy,Portugal,4,3,Italy
5,A,Italy,Belgium,4,6,Belgium
6,A,Belgium,Portugal,4,3,Belgium
7,B,Spain,England,6,2,Spain
8,B,Spain,Germany,4,3,Spain
9,B,Spain,Netherlands,4,5,Netherlands
10,B,Netherlands,England,1,3,England


In [0]:
%sql
UPDATE matches
SET winner_country = 
    CASE
        WHEN home_country_goal > away_country_goal THEN home_country
        WHEN home_country_goal < away_country_goal THEN away_country
        ELSE 
        CASE 
                WHEN (home_country_goal + 1) % 2 = 1 THEN home_country
                ELSE away_country
            END
    END
     

num_affected_rows
48


In [0]:
%sql
select * from matches order by MatchId;

MatchId,TeamGroup,home_country,away_country,home_country_goal,away_country_goal,winner_country
1,A,France,Portugal,3,4,Portugal
2,A,France,Belgium,3,3,Belgium
3,A,France,Italy,5,1,France
4,A,Italy,Portugal,4,3,Italy
5,A,Italy,Belgium,4,6,Belgium
6,A,Belgium,Portugal,4,3,Belgium
7,B,Spain,England,6,2,Spain
8,B,Spain,Germany,4,3,Spain
9,B,Spain,Netherlands,4,5,Netherlands
10,B,Netherlands,England,1,3,England


Semi Final

In [0]:
%sql
INSERT INTO matches (MatchID, TeamGroup, home_country, away_country, home_country_goal, away_country_goal, winner_country)

SELECT
row_number() OVER (ORDER BY 1) + 48 AS MatchId,
'N' AS Team_group,
r1.winner_country AS home_country,
r2.winner_country AS away_country,
CAST(rand() * 6 AS INTEGER) AS home_country_goal,
CAST(rand() * 6 AS INTEGER) AS away_country_goal,
'None' AS match_result

FROM 
    matches r1
JOIN 
    matches r2 ON 
        r1.MatchId = 45 AND r2.MatchId = 46

UNION ALL

SELECT
row_number() OVER (ORDER BY 1) + 49 AS MatchId,
'N' AS Team_group,
r1.winner_country AS home_country,
r2.winner_country AS away_country,
CAST(rand() * 6 AS INTEGER) AS home_country_goal,
CAST(rand() * 6 AS INTEGER) AS away_country_goal,
'None' AS match_result

FROM 
    matches r1
JOIN 
    matches r2 ON 
        r1.MatchId = 47 AND r2.MatchId = 48

num_affected_rows,num_inserted_rows
2,2


In [0]:
%sql
UPDATE matches
SET winner_country = 
    CASE
        WHEN home_country_goal > away_country_goal THEN home_country
        WHEN home_country_goal < away_country_goal THEN away_country
        ELSE 
        CASE 
                WHEN (home_country_goal + 1) % 2 = 1 THEN home_country
                ELSE away_country
            END
    END

num_affected_rows
50


In [0]:
%sql
select * from matches order by MatchId;

MatchId,TeamGroup,home_country,away_country,home_country_goal,away_country_goal,winner_country
1,A,France,Portugal,3,4,Portugal
2,A,France,Belgium,3,3,Belgium
3,A,France,Italy,5,1,France
4,A,Italy,Portugal,4,3,Italy
5,A,Italy,Belgium,4,6,Belgium
6,A,Belgium,Portugal,4,3,Belgium
7,B,Spain,England,6,2,Spain
8,B,Spain,Germany,4,3,Spain
9,B,Spain,Netherlands,4,5,Netherlands
10,B,Netherlands,England,1,3,England


Third Place

In [0]:
%sql
INSERT INTO matches (MatchID, TeamGroup, home_country, away_country, home_country_goal, away_country_goal, winner_country)
WITH losing_teams AS (
    SELECT
        CASE
            WHEN winner_country != home_country THEN home_country
            ELSE away_country
        END AS losing_teams
    FROM
        matches
    WHERE MatchId IN (49,50)
)

SELECT 
    Row_Number() OVER(ORDER BY 1) + 50 AS match_id,
    'N' as GroupName,
    T1.losing_teams AS home_country,
    T2.losing_teams AS away_country,
    CAST(rand() * 6 AS INTEGER) AS home_country_goal, 
    CAST(rand() * 6 AS INTEGER) AS away_country_goal,
    'None' AS match_results
FROM 
    losing_teams T1
JOIN 
    losing_teams T2 ON T1.losing_teams < T2.losing_teams;

num_affected_rows,num_inserted_rows
1,1


In [0]:
%sql
UPDATE matches
SET winner_country = 
    CASE
        WHEN home_country_goal > away_country_goal THEN home_country
        WHEN home_country_goal < away_country_goal THEN away_country
        ELSE 
        CASE 
                WHEN (home_country_goal + 1) % 2 = 1 THEN home_country
                ELSE away_country
            END
    END

num_affected_rows
51


In [0]:
%sql
select * from matches order by MatchId;

MatchId,TeamGroup,home_country,away_country,home_country_goal,away_country_goal,winner_country
1,A,France,Portugal,3,4,Portugal
2,A,France,Belgium,3,3,Belgium
3,A,France,Italy,5,1,France
4,A,Italy,Portugal,4,3,Italy
5,A,Italy,Belgium,4,6,Belgium
6,A,Belgium,Portugal,4,3,Belgium
7,B,Spain,England,6,2,Spain
8,B,Spain,Germany,4,3,Spain
9,B,Spain,Netherlands,4,5,Netherlands
10,B,Netherlands,England,1,3,England


Final

In [0]:
%sql
INSERT INTO matches (MatchID, TeamGroup, home_country, away_country, home_country_goal, away_country_goal, winner_country)

SELECT
row_number() OVER (ORDER BY 1) + 51 AS MatchId,
'N' AS Team_group,
r1.winner_country AS home_country,
r2.winner_country AS away_country,
CAST(rand() * 6 AS INTEGER) AS home_country_goal,
CAST(rand() * 6 AS INTEGER) AS away_country_goal,
'None' AS match_result

FROM 
    matches r1
JOIN 
    matches r2 ON 
        r1.MatchId = 49 AND r2.MatchId = 50


num_affected_rows,num_inserted_rows
1,1


In [0]:
%sql
UPDATE matches
SET winner_country = 
    CASE
        WHEN home_country_goal > away_country_goal THEN home_country
        WHEN home_country_goal < away_country_goal THEN away_country
        ELSE 
        CASE 
                WHEN (home_country_goal + 1) % 2 = 1 THEN home_country
                ELSE away_country
            END
    END

num_affected_rows
52


In [0]:
%sql
select * from matches order by MatchId;

MatchId,TeamGroup,home_country,away_country,home_country_goal,away_country_goal,winner_country
1,A,France,Portugal,3,4,Portugal
2,A,France,Belgium,3,3,Belgium
3,A,France,Italy,5,1,France
4,A,Italy,Portugal,4,3,Italy
5,A,Italy,Belgium,4,6,Belgium
6,A,Belgium,Portugal,4,3,Belgium
7,B,Spain,England,6,2,Spain
8,B,Spain,Germany,4,3,Spain
9,B,Spain,Netherlands,4,5,Netherlands
10,B,Netherlands,England,1,3,England


In [0]:
%sql
SELECT
    *
FROM
    (
        SELECT
            ROW_NUMBER () OVER (ORDER BY MatchId) AS RowNum,
            MatchId, home_country, away_country, home_country_goal, away_country_goal, winner_country
        FROM
            matches
    ) sub

RowNum,MatchId,home_country,away_country,home_country_goal,away_country_goal,winner_country
1,1,France,Portugal,3,4,Portugal
2,2,France,Belgium,3,3,Belgium
3,3,France,Italy,5,1,France
4,4,Italy,Portugal,4,3,Italy
5,5,Italy,Belgium,4,6,Belgium
6,6,Belgium,Portugal,4,3,Belgium
7,7,Spain,England,6,2,Spain
8,8,Spain,Germany,4,3,Spain
9,9,Spain,Netherlands,4,5,Netherlands
10,10,Netherlands,England,1,3,England


In [0]:
%python
import pandas as pd
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Eurocup").getOrCreate()


In [0]:
%python
all_teams_stats = spark.sql("SELECT * from group_stage_ranking")
round_of_16 = spark.sql("""
                        SELECT 
    CASE WHEN MatchID IN (37,38,39,40) THEN 'left' ELSE 'right' END AS side,
    MatchID,
    home_country, 
    away_country, 
    home_country_goal, 
    away_country_goal 
FROM 
    matches 
WHERE 
    MatchID IN (37,38,39,40,41,42,43,44) 
    AND MatchId In (37, 38, 39, 40, 41, 42, 43, 44)
ORDER BY 
    CASE 
        WHEN MatchID = 39 THEN 1
        WHEN MatchID = 37 THEN 2
        WHEN MatchID = 41 THEN 3
        WHEN MatchID = 42 THEN 4
        WHEN MatchID = 43 THEN 5
        WHEN MatchID = 44 THEN 6
        WHEN MatchID = 40 THEN 7
        WHEN MatchID = 38 THEN 8
    END;

                        """)
quarter_final = spark.sql("SELECT home_country, away_country, home_country_goal, away_country_goal FROM matches WHERE MatchID IN (45,46,47,48)  ORDER BY MatchID")
semi_final = spark.sql("SELECT home_country, away_country, home_country_goal, away_country_goal FROM matches WHERE MatchID IN (50,51) ORDER BY MatchID DESC")
final = spark.sql("SELECT home_country, away_country, home_country_goal, away_country_goal FROM matches WHERE MatchID = 52 ORDER BY MatchID")
third_place = spark.sql("SELECT MatchID FROM matches WHERE MatchID = 49 ORDER BY MatchID")

In [0]:
%python

all_teams_stats_df = all_teams_stats.toPandas()
round_of_16_df = round_of_16.toPandas()
quarter_final_df = quarter_final.toPandas()
semi_final_df = semi_final.toPandas()
final_df = final.toPandas()
third_place_df = third_place.toPandas()
     

In [0]:
%python
table_visualization = pd.DataFrame({
    'Teams': all_teams_stats_df['TeamName'],
    'Group': all_teams_stats_df['TeamGroup'],
    'W': all_teams_stats_df['Matches_won'],
    'D': all_teams_stats_df['Matches_draw'],
    'L': all_teams_stats_df['Matches_lost'],
    'GF': all_teams_stats_df['Goals_scored'],
    'GA': all_teams_stats_df['Goals_allowed'],
    'GD': all_teams_stats_df['Goal_difference'],
    'Pts': all_teams_stats_df['points']
})

# Create blank row as a dictionary to add one blank row for each group
blank_row = {
    'Teams': '', 
    'Group': '', 
    'W': '',
    'D': '',
    'L': '',
    'GF': '',
    'GA': '',
    'GD': '',
    'Pts': ''
}

groups = []
for i in range(0, len(table_visualization), 4):
    group = table_visualization.iloc[i:i+4]  
    groups.append(group) 
    if i < len(table_visualization) - 4:
        groups.append(pd.DataFrame([blank_row])) 

visualization_with_space = pd.concat(groups).reset_index(drop=True)
half_table_index = len(visualization_with_space) // 2
spark.conf.set("spark.sql.execution.arrow.pyspark.enabled", "false")
display(visualization_with_space)
print(half_table_index)

Teams,Group,W,D,L,GF,GA,GD,Pts
Belgium,A,2,1,0,13,10,3,7
France,A,1,1,1,11,8,3,4
Italy,A,1,0,2,9,14,-5,3
Portugal,A,1,0,2,10,11,-1,3
,,,,,,,,
Spain,B,2,0,1,14,10,4,6
England,B,2,0,1,10,11,-1,6
Netherlands,B,1,0,2,8,12,-4,3
Germany,B,1,0,2,12,11,1,3
,,,,,,,,


14


In [0]:
%python
#Table information for the Group A,B,C
left_table = pd.DataFrame(columns=['Teams','Group','W','D','L','GF','GA','GD','Pts'])
left_table = pd.concat([left_table, pd.DataFrame([blank_row])], ignore_index=True)
left_table = pd.concat([left_table, visualization_with_space.iloc[:half_table_index]]).reset_index(drop=True)

#Table information for the group C,D,E
right_table = pd.DataFrame(columns=['Teams','Group','W','D','L','GF','GA','GD','Pts'])
right_table = pd.concat([right_table, visualization_with_space.iloc[half_table_index +1:]]).reset_index(drop=True)
right_table = pd.concat([pd.DataFrame([blank_row]), right_table], ignore_index=True)

display(left_table)
display(right_table)

Teams,Group,W,D,L,GF,GA,GD,Pts
,,,,,,,,
Belgium,A,2,1,0,13,10,3,7
France,A,1,1,1,11,8,3,4
Italy,A,1,0,2,9,14,-5,3
Portugal,A,1,0,2,10,11,-1,3
,,,,,,,,
Spain,B,2,0,1,14,10,4,6
England,B,2,0,1,10,11,-1,6
Netherlands,B,1,0,2,8,12,-4,3
Germany,B,1,0,2,12,11,1,3


Teams,Group,W,D,L,GF,GA,GD,Pts
,,,,,,,,
Ukraine,D,2,1,0,13,8,5,7
Poland,D,1,2,0,11,7,4,5
Sweden,D,1,1,1,9,7,2,4
Croatia,D,0,0,3,4,15,-11,0
,,,,,,,,
Czech Republic,E,2,0,1,10,6,4,6
Slovakia,E,2,0,1,8,8,0,6
Wales,E,1,0,2,7,11,-4,3
Hungary,E,1,0,2,11,11,0,3


In [0]:
%python
# Create an empty DataFrame for round of sixteen
visualize_r16 = pd.DataFrame(columns=['Round_of_16', 'rest'])

for _ in range(2):
    visualize_r16 = pd.concat([visualize_r16, pd.DataFrame({'Round_of_16': ['']})], ignore_index=True)

for index, row in round_of_16_df.iterrows():
    visualize_r16 = pd.concat([visualize_r16, pd.DataFrame({'Round_of_16': [f"{row['home_country']} ({row['home_country_goal']})"]})], ignore_index=True)
    visualize_r16 = pd.concat([visualize_r16, pd.DataFrame({'Round_of_16': [f"{row['away_country']} ({row['away_country_goal']})"]})], ignore_index=True)

    for _ in range(2):
        visualize_r16 = pd.concat([visualize_r16, pd.DataFrame({'Round_of_16': ['']})], ignore_index=True)

visualize_r16['rest'] = ''

splited_r16_index = len(visualize_r16) // 2
print(splited_r16_index)
display(visualize_r16)

17


Round_of_16,rest
,
,
Spain (2),
Portugal (0),
,
,
Belgium (0),
Austria (3),
,
,


In [0]:
%python
left_side_r16 = visualize_r16.iloc[:splited_r16_index + 1].reset_index(drop = True)
right_side_r16 = visualize_r16.iloc[splited_r16_index - 1:].reset_index(drop = True)
display(left_side_r16)
display(right_side_r16)
     

Round_of_16,rest
,
,
Spain (2),
Portugal (0),
,
,
Belgium (0),
Austria (3),
,
,


Round_of_16,rest
,
,
Czech Republic (1),
Germany (2),
,
,
Poland (2),
Scotland (2),
,
,


In [0]:
%python
# Dataframe for quarter final
visualize_qf = pd.DataFrame(columns=['quarter_final'])

for _ in range(4):
    visualize_qf = pd.concat([visualize_qf, pd.DataFrame({'quarter_final': ['']})], ignore_index=True)

for index, row in quarter_final_df.iterrows():
    visualize_qf = pd.concat([visualize_qf, pd.DataFrame({'quarter_final': [f"{row['home_country']} ({row['home_country_goal']})"]})], ignore_index=True)
    visualize_qf = pd.concat([visualize_qf, pd.DataFrame({'quarter_final': [f"{row['away_country']} ({row['away_country_goal']})"]})], ignore_index=True)

    for _ in range(6):
        visualize_qf = pd.concat([visualize_qf, pd.DataFrame({'quarter_final': ['']})], ignore_index=True)
visualize_qf['rest'] = ''
splited_qf_index = len(visualize_qf) // 2
display(visualize_qf)


quarter_final,rest
,
,
,
,
Austria (0),
England (0),
,
,
,
,


In [0]:
%python
left_side_qf = pd.DataFrame(columns=['Quater_Final'])
left_side_qf = pd.concat([left_side_qf,visualize_qf.iloc[:splited_qf_index]]).reset_index(drop=True)
right_side_qf = pd.DataFrame(columns=['Quater_Final'])
right_side_qf = pd.concat([right_side_qf,visualize_qf.iloc[splited_qf_index-2:]]).reset_index(drop=True)


display(left_side_qf)
display(right_side_qf)

Quater_Final,quarter_final,rest
NaN,,
NaN,,
NaN,,
NaN,,
NaN,Austria (0),
NaN,England (0),
NaN,,
NaN,,
NaN,,
NaN,,


Quater_Final,quarter_final,rest
NaN,,
NaN,,
NaN,,
NaN,,
NaN,Finland (5),
NaN,Slovakia (3),
NaN,,
NaN,,
NaN,,
NaN,,


In [0]:
%python

import pandas as pd

visualize_sf = pd.DataFrame(columns=['semi_final'])

for _ in range(8):
    visualize_sf = pd.concat([visualize_sf, pd.DataFrame({'semi_final': ['']})], ignore_index=True)

for index, row in semi_final_df.iterrows():
    visualize_sf = pd.concat([visualize_sf, pd.DataFrame({'semi_final': [f"{row['home_country']} ({row['home_country_goal']})"]})], ignore_index=True)
    visualize_sf = pd.concat([visualize_sf, pd.DataFrame({'semi_final': [f"{row['away_country']} ({row['away_country_goal']})"]})], ignore_index=True)
    for _ in range(6):
        visualize_sf = pd.concat([visualize_sf, pd.DataFrame({'semi_final': ['']})], ignore_index=True)
visualize_sf['rest'] = ''
splited_sf_index = len(visualize_sf) // 2
print(splited_sf_index)
display(visualize_sf)

12


semi_final,rest
,
,
,
,
,
,
,
,
Austria (2),
Poland (5),


In [0]:
%python
left_side_sf = pd.DataFrame(columns=['Semi_Final'])
left_side_sf = pd.concat([left_side_sf, visualize_sf.iloc[:splited_sf_index]]).reset_index(drop=True)

for _ in range(5):
    left_side_sf = pd.concat([left_side_sf, pd.DataFrame({'Semi_Final': [''], 'rest': ['']})], ignore_index=True)

right_side_sf = pd.DataFrame({'Semi_Final': [''] * 7, 'rest': [''] * 7})
right_side_sf = pd.concat([right_side_sf, visualize_sf.iloc[splited_sf_index:]]).reset_index(drop=True)

display(left_side_sf)
display(right_side_sf)

Semi_Final,semi_final,rest
NaN,,
NaN,,
NaN,,
NaN,,
NaN,,
NaN,,
NaN,,
NaN,,
NaN,Austria (2),
NaN,Poland (5),


Semi_Final,rest,semi_final
,,NaN
,,NaN
,,NaN
,,NaN
,,NaN
,,NaN
,,NaN
NaN,,
NaN,,
NaN,,


In [0]:
%python

visualize_final = pd.DataFrame(columns=['Final'])
for _ in range(6):
    visualize_final = pd.concat([visualize_final, pd.DataFrame({'Final': ['']})], ignore_index=True)

for index,row in final_df.iterrows():
    visualize_final = pd.concat([visualize_final, pd.DataFrame({'Final': [f"{row['home_country']} ({row['home_country_goal']})"]})], ignore_index=True)
    visualize_final = pd.concat([visualize_final, pd.DataFrame({'Final': [f"{row['away_country']} ({row['away_country_goal']})"]})], ignore_index=True)

    if (index + 1) %2 == 0:
        visualize_final = pd.concat([visualize_final, pd.DataFrame({'Final': ['']})], ignore_index=True)
        visualize_final = pd.concat([visualize_final, pd.DataFrame({'Final': ['']})], ignore_index=True)
    for _ in range(2):
        visualize_final = pd.concat([visualize_final, pd.DataFrame({'Final': ['']})], ignore_index=True)
visualize_final['space'] = ''
display(visualize_final)

Final,space
,
,
,
,
,
,
Sweden (0),
Finland (5),
,
,


In [0]:
%python
left_side_qf.columns = [f"{col}q2" for col in left_side_qf.columns]

merged_df = pd.concat([left_side_r16, left_side_qf], axis=1)

left_side_sf.columns = [f"{col}s1" for col in left_side_sf.columns]
merged_df = pd.concat([merged_df, left_side_sf], axis=1)

visualize_final.columns = [f"{col}f" for col in visualize_final.columns]
merged_df = pd.concat([merged_df, visualize_final], axis=1)

right_side_sf.columns = [f"{col}s2" for col in right_side_sf.columns]

merged_df = pd.concat([merged_df, right_side_sf], axis=1)



right_side_qf.columns = [f"{col}q1" for col in right_side_qf.columns]
# Concatenate the DataFrames
merged_df = pd.concat([merged_df, right_side_qf], axis=1)

right_side_r16.columns = [f"{col}r2" for col in right_side_r16.columns]

# Concatenate the DataFrames
merged_df = pd.concat([merged_df,right_side_r16], axis=1)

left_table.columns = [f"{col}g1" for col in left_table.columns]
right_table.columns = [f"{col}g2" for col in right_table.columns]

# Concatenate the DataFrames
merged_df = pd.concat([left_table,merged_df, right_table], axis=1)

merged_df.fillna('', inplace=True)

display(merged_df)

Teamsg1g1,Groupg1g1,Wg1g1,Dg1g1,Lg1g1,GFg1g1,GAg1g1,GDg1g1,Ptsg1g1,Round_of_16,rest,Quater_Finalq2q2,quarter_finalq2q2,restq2q2,Semi_Finals1s1,semi_finals1s1,rests1s1,Finalff,spaceff,Semi_Finals2s2,rests2s2,semi_finals2s2,Quater_Finalq1q1,quarter_finalq1q1,restq1q1,Round_of_16r2r2,restr2r2,Teamsg2g2,Groupg2g2,Wg2g2,Dg2g2,Lg2g2,GFg2g2,GAg2g2,GDg2g2,Ptsg2g2
,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
Belgium,A,2,1,0,13,10,3,7,,,,,,,,,,,,,,,,,,,Ukraine,D,2,1,0,13,8,5,7
France,A,1,1,1,11,8,3,4,Spain (2),,,,,,,,,,,,,,,,Czech Republic (1),,Poland,D,1,2,0,11,7,4,5
Italy,A,1,0,2,9,14,-5,3,Portugal (0),,,,,,,,,,,,,,,,Germany (2),,Sweden,D,1,1,1,9,7,2,4
Portugal,A,1,0,2,10,11,-1,3,,,,Austria (0),,,,,,,,,,,Finland (5),,,,Croatia,D,0,0,3,4,15,-11,0
,,,,,,,,,,,,England (0),,,,,,,,,,,Slovakia (3),,,,,,,,,,,,
Spain,B,2,0,1,14,10,4,6,Belgium (0),,,,,,,,Sweden (0),,,,,,,,Poland (2),,Czech Republic,E,2,0,1,10,6,4,6
England,B,2,0,1,10,11,-1,6,Austria (3),,,,,,,,Finland (5),,,,,,,,Scotland (2),,Slovakia,E,2,0,1,8,8,0,6
Netherlands,B,1,0,2,8,12,-4,3,,,,,,,Austria (2),,,,,,,,,,,,Wales,E,1,0,2,7,11,-4,3
Germany,B,1,0,2,12,11,1,3,,,,,,,Poland (5),,,,,,,,,,,,Hungary,E,1,0,2,11,11,0,3
